In [1]:
! pip install --upgrade gspread
! pip install --upgrade gspread-pandas
! pip install --upgrade gspread-dataframe

  Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1
  Found existing installation: gspread-dataframe 3.0.8
    Uninstalling gspread-dataframe-3.0.8:
      Successfully uninstalled gspread-dataframe-3.0.8


In [2]:
import keras
import numpy as np
import pandas as pd
from keras.layers import Input, Dense, Flatten, Dropout
from keras.models import Model
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.utils import to_categorical
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from google.colab import auth
auth.authenticate_user()
import gspread
import gspread_dataframe 
from oauth2client.client import GoogleCredentials

# import BatchNormalization
from keras.layers.normalization import BatchNormalization

In [3]:
def plot_acc(history, folder, name):
  '''Lưu lại biểu đồ acc và loss của sau khi train'''
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']

  loss=history.history['loss']
  val_loss=history.history['val_loss']

  epochs_range = range(len(acc))

  plt.figure(figsize=(8, 8))
  plt.subplot(1, 2, 1)
  plt.plot(epochs_range, acc, label='Training Accuracy')
  plt.plot(epochs_range, val_acc, label='Validation Accuracy')
  plt.legend(loc='lower right')
  plt.title('Training and Validation Accuracy')

  plt.subplot(1, 2, 2)
  plt.plot(epochs_range, loss, label='Training Loss')
  plt.plot(epochs_range, val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.title('Training and Validation Loss')
  plt.savefig('/content/drive/MyDrive/Histogram/{}/{}.png'.format(folder, name))

  plt.close('all')

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, multilabel_confusion_matrix

def get_report(y_true, y_pred):
  cnf_matrix = confusion_matrix(y_true, y_pred)
  fp = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
  fn = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
  tp = np.diag(cnf_matrix)
  tn = cnf_matrix.sum() - (fp + fn + tp)
  fb = fp.astype(float)
  fn = fn.astype(float)
  tp = tp.astype(float)
  tn = tn.astype(float)
  report = classification_report(y_true, y_pred, output_dict=True)
  return (cnf_matrix, tn, fp, fn, tp, report)

In [4]:
files = []
files.append(np.array(pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/file-0.csv", header=None, skiprows=1)))
files.append(np.array(pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/file-1.csv", header=None, skiprows=1)))
files.append(np.array(pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/file-2.csv", header=None, skiprows=1)))
files.append(np.array(pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/file-3.csv", header=None, skiprows=1)))
files.append(np.array(pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/file-4.csv", header=None, skiprows=1)))
files.append(np.array(pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/file-5.csv", header=None, skiprows=1)))
files.append(np.array(pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/file-6.csv", header=None, skiprows=1)))
files.append(np.array(pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/file-7.csv", header=None, skiprows=1)))
files.append(np.array(pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/file-8.csv", header=None, skiprows=1)))
files.append(np.array(pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/file-9.csv", header=None, skiprows=1)))

In [ ]:
gc = gspread.authorize(GoogleCredentials.get_application_default())
sh = gc.create('AMD_DNN_image256x256_per_api')
for index in range(3,10):
  train = np.concatenate([files[i%10] for i in range(index + 2, index + 10)], axis=0)
  test = files[(index+1)%10]
  val = files[index]

  x_train = train[:, 3:].astype(np.float32)
  y_train = train[:, 2].astype('int')
  y_train = to_categorical(y_train, 228)
  del train

  x_val = val[:, 3:].astype(np.float32)
  y_val = val[:, 2].astype('int')
  y_val = to_categorical(y_val, 228)
  del val

  x_test = test[:, 3:].astype(np.float32)
  y_test = test[:, 2].astype('int')
  del test

  input = Input(shape=(x_val.shape[1],))
  # dense1 = Dense(x_val.shape[1], input_shape=(x_val.shape[1],), activation="relu")
  batch1 = BatchNormalization()(input)
  hidden1 = Dense(16300, activation='relu')(batch1)
  dropout1 = Dropout(0.5)(hidden1)

  batch2 = BatchNormalization()(dropout1)
  hidden2 = Dense(14600, activation='relu')(batch2)
  dropout2 = Dropout(0.5)(hidden2)

  batch3 = BatchNormalization()(dropout2)
  hidden3 = Dense(12900, activation='relu')(batch3)
  dropout3 = Dropout(0.5)(hidden3)

  batch4 = BatchNormalization()(dropout3)
  hidden4 = Dense(11200, activation='relu')(batch4)
  dropout4 = Dropout(0.5)(hidden4)

  batch5 = BatchNormalization()(dropout4)
  hidden5 = Dense(9500, activation='relu')(batch5)
  dropout5 = Dropout(0.5)(hidden5)

  batch6 = BatchNormalization()(dropout5)
  hidden6 = Dense(7800, activation='relu')(batch6)
  dropout6 = Dropout(0.5)(hidden6)

  batch7 = BatchNormalization()(dropout6)
  hidden7 = Dense(6100, activation='relu')(batch7)
  dropout7 = Dropout(0.5)(hidden7)

  batch8 = BatchNormalization()(dropout7)
  hidden8 = Dense(4400, activation='relu')(batch8)
  dropout8 = Dropout(0.5)(hidden8)

  batch9 = BatchNormalization()(dropout8)
  hidden9 = Dense(2700, activation='relu')(batch9)
  dropout9 = Dropout(0.5)(hidden9)

  batch10 = BatchNormalization()(dropout9)
  hidden10 = Dense(1000, activation='relu')(batch10)
  dropout10 = Dropout(0.5)(hidden10)

  batch = BatchNormalization()(dropout10)
  output = Dense(228, activation='softmax')(batch)

  model = Model(inputs=input, outputs=output)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  history = model.fit(x_train, y_train, epochs=13, batch_size = 32 ,validation_data=(x_val, y_val))

  y_pred_DNN = model.predict(x_test)
  y_pred = np.argmax(y_pred_DNN, axis=-1)
  #draw acc loss
  plot_acc(history, 'DNN_AMD_256perAPI', 'acc_train{}'.format(index))

  (cnf_matrix, tn, fp, fn, tp, report) = get_report(list(y_test), list(y_pred))
  #create sheet report
  worksheet_th4 = sh.add_worksheet(title="report_train{}".format(index), rows="100", cols="20")
  df = pd.DataFrame(report).transpose()
  gspread_dataframe.set_with_dataframe(worksheet_th4, df, include_index =True)  
  #create sheet confusion matrix
  labels = list(np.unique(list(y_test) + list(y_pred)))
  cnf_matrix = pd.DataFrame(cnf_matrix, index=labels, columns=labels, dtype=float)
  worksheet_th4_cnf = sh.add_worksheet(title="cnf_train{}".format(index), rows="100", cols="100")
  gspread_dataframe.set_with_dataframe(worksheet_th4_cnf, cnf_matrix, include_index =True)
  #create sheet tn, tp, fp,fn
  acc = (tp + tn) / (tp + tn + fp + fn)
  tp = tp.reshape((tp.shape[0], 1))
  tn = tn.reshape((tn.shape[0], 1))
  fp = fp.reshape((fp.shape[0], 1))
  fn = fn.reshape((fn.shape[0], 1))
  acc = acc.reshape((tp.shape[0], 1))
  labels = (np.array(labels))
  labels = labels.reshape((labels.shape[0], 1))
  table = np.concatenate((labels, tp, tn, fp, fn, acc), axis=1)
  table = pd.DataFrame(table, columns=['Label', 'TP', 'TN', 'FP', 'FN', 'ACC'], dtype=float)
  worksheet_th4_table = sh.add_worksheet(title="table_train{}".format(index), rows="100", cols="100")
  gspread_dataframe.set_with_dataframe(worksheet_th4_table, table, include_index =True) 

Epoch 1/13
652/652 [==============================] - 1677s 3s/step - loss: 2.8081 - accuracy: 0.3925 - val_loss: 2.3668 - val_accuracy: 0.7043
Epoch 2/13
652/652 [==============================] - 1675s 3s/step - loss: 1.0863 - accuracy: 0.7283 - val_loss: 1.3681 - val_accuracy: 0.8061
Epoch 3/13
652/652 [==============================] - 1675s 3s/step - loss: 0.7595 - accuracy: 0.8186 - val_loss: 0.8124 - val_accuracy: 0.8526
Epoch 4/13
652/652 [==============================] - 1675s 3s/step - loss: 0.6138 - accuracy: 0.8502 - val_loss: 1.0490 - val_accuracy: 0.8231
Epoch 5/13
652/652 [==============================] - 1677s 3s/step - loss: 0.5279 - accuracy: 0.8716 - val_loss: 0.8452 - val_accuracy: 0.8611
Epoch 6/13
652/652 [==============================] - 1673s 3s/step - loss: 0.5067 - accuracy: 0.8718 - val_loss: 0.6875 - val_accuracy: 0.8854
Epoch 7/13
652/652 [==============================] - 1677s 3s/step - loss: 0.4597 - accuracy: 0.8832 - val_loss: 0.9244 - val_accuracy:

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/13
653/653 [==============================] - 1644s 3s/step - loss: 2.8362 - accuracy: 0.3861 - val_loss: 2.3622 - val_accuracy: 0.7340
Epoch 2/13
653/653 [==============================] - 1633s 3s/step - loss: 1.1183 - accuracy: 0.7243 - val_loss: 0.9975 - val_accuracy: 0.8238
Epoch 3/13
653/653 [==============================] - 1635s 3s/step - loss: 0.7629 - accuracy: 0.8237 - val_loss: 0.8789 - val_accuracy: 0.8622
Epoch 4/13
653/653 [==============================] - 1636s 3s/step - loss: 0.5807 - accuracy: 0.8641 - val_loss: 0.7806 - val_accuracy: 0.8773
Epoch 5/13
653/653 [==============================] - 1634s 3s/step - loss: 0.4971 - accuracy: 0.8803 - val_loss: 0.4969 - val_accuracy: 0.8997
Epoch 6/13
653/653 [==============================] - 1634s 3s/step - loss: 0.4364 - accuracy: 0.8883 - val_loss: 0.6745 - val_accuracy: 0.8955
Epoch 7/13
653/653 [==============================] - 1639s 3s/step - loss: 0.4003 - accuracy: 0.8977 - val_loss: 0.8844 - val_accuracy:

TH1: Input: image+per+api tạo thành vector đưa vào các lớp ẩn, droput, batch normalization như trong bài báo

In [ ]:
index = 1
train = np.concatenate([files[i%10] for i in range(index + 2, index + 10)], axis=0)
test = files[(index+1)%10]
val = files[index]

x_train = train[:, 3:].astype(np.float32)
y_train = train[:, 2].astype('int')
# y_train = to_categorical(y_train, 228)
del train

x_val = val[:, 3:].astype(np.float32)
y_val = val[:, 2].astype('int')
# y_val = to_categorical(y_val, 228)
del val

x_test = test[:, 3:].astype(np.float32)
y_test = test[:, 2].astype('int')
del test

In [ ]:
input = Input(shape=(x_val.shape[1],))
# dense1 = Dense(x_val.shape[1], input_shape=(x_val.shape[1],), activation="relu")
batch1 = BatchNormalization()(input)
hidden1 = Dense(16300, activation='relu')(batch1)
dropout1 = Dropout(0.5)(hidden1)

batch2 = BatchNormalization()(dropout1)
hidden2 = Dense(14600, activation='relu')(batch2)
dropout2 = Dropout(0.5)(hidden2)

batch3 = BatchNormalization()(dropout2)
hidden3 = Dense(12900, activation='relu')(batch3)
dropout3 = Dropout(0.5)(hidden3)

batch4 = BatchNormalization()(dropout3)
hidden4 = Dense(11200, activation='relu')(batch4)
dropout4 = Dropout(0.5)(hidden4)

batch5 = BatchNormalization()(dropout4)
hidden5 = Dense(9500, activation='relu')(batch5)
dropout5 = Dropout(0.5)(hidden5)

batch6 = BatchNormalization()(dropout5)
hidden6 = Dense(7800, activation='relu')(batch6)
dropout6 = Dropout(0.5)(hidden6)

batch7 = BatchNormalization()(dropout6)
hidden7 = Dense(6100, activation='relu')(batch7)
dropout7 = Dropout(0.5)(hidden7)

batch8 = BatchNormalization()(dropout7)
hidden8 = Dense(4400, activation='relu')(batch8)
dropout8 = Dropout(0.5)(hidden8)

batch9 = BatchNormalization()(dropout8)
hidden9 = Dense(2700, activation='relu')(batch9)
dropout9 = Dropout(0.5)(hidden9)

batch10 = BatchNormalization()(dropout9)
hidden10 = Dense(1000, activation='relu')(batch10)
dropout10 = Dropout(0.5)(hidden10)

batch = BatchNormalization()(dropout10)
output = Dense(228, activation='softmax')(batch)

model = Model(inputs=input, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=20, batch_size = 128 ,validation_data=(x_val, y_val))

Epoch 1/20
163/163 [==============================] - 431s 3s/step - loss: 3.3133 - accuracy: 0.3641 - val_loss: 44.1826 - val_accuracy: 0.0870
Epoch 2/20
163/163 [==============================] - 430s 3s/step - loss: 1.0235 - accuracy: 0.7386 - val_loss: 1.1797 - val_accuracy: 0.7911
Epoch 3/20
163/163 [==============================] - 433s 3s/step - loss: 0.6282 - accuracy: 0.8473 - val_loss: 0.7060 - val_accuracy: 0.8634
Epoch 4/20
163/163 [==============================] - 430s 3s/step - loss: 0.4568 - accuracy: 0.8871 - val_loss: 0.5923 - val_accuracy: 0.8859
Epoch 5/20
163/163 [==============================] - 430s 3s/step - loss: 0.3658 - accuracy: 0.9077 - val_loss: 0.6170 - val_accuracy: 0.8894
Epoch 6/20
163/163 [==============================] - 432s 3s/step - loss: 0.3174 - accuracy: 0.9178 - val_loss: 0.4373 - val_accuracy: 0.9072
Epoch 7/20
163/163 [==============================] - 430s 3s/step - loss: 0.2611 - accuracy: 0.9302 - val_loss: 0.4908 - val_accuracy: 0.906

In [ ]:
y_pred_DNN = model.predict(x_test)
y_pred = np.argmax(y_pred_DNN, axis=-1)
# report = classification_report(list(y_test), list(y_pred), output_dict=True)
report = classification_report(list(y_test), list(y_pred))
print(report)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       673
          18       1.00      1.00      1.00         4
          33       1.00      0.96      0.98        54
          41       1.00      1.00      1.00         1
          47       0.50      1.00      0.67         1
          49       0.33      1.00      0.50         1
          56       0.00      0.00      0.00         1
          87       1.00      0.33      0.50         3
         104       0.88      1.00      0.93         7
         110       0.00      0.00      0.00         1
         112       0.00      0.00      0.00         1
         148       0.00      0.00      0.00         1
         155       1.00      1.00      1.00         2
         157       0.00      0.00      0.00         1
         161       0.00      0.00      0.00         1
         171       0.67      1.00      0.80         2
         173       1.00      0.50      0.67         2
         178       0.50    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
